In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers, models
from keras.models import Model
from keras.layers import Input, Dense, Conv2D,Conv1D, MaxPooling2D, UpSampling2D,Activation,GlobalAveragePooling2D,AveragePooling2D,BatchNormalization,Dropout,Flatten 
from tensorflow.keras.layers import concatenate,Add, Lambda
from keras.utils import plot_model
from tensorflow.keras.optimizers import schedules, SGD
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

2024-07-11 14:18:43.356059: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import numpy as np
import os
import json
import sys
import warnings

CNN

In [3]:
def convolutional_block(x, filters,block_id=0,stack_id=0, kernel_size=3, stride=1):
    y = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    y = Activation('relu')(y)
    y = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(y)
    y = Activation('relu')(y)
    return y
def creation_model(input_shape, depth, num_classes=2):
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    num_filters = 16
    num_blocks = int((depth - 2) / 6)
    inputs = Input(shape=input_shape)
    x = Conv2D(num_filters, kernel_size=3, strides=1, padding='same')(inputs)
    for stack in range(3):
        for block_id in range(num_blocks):
           x = convolutional_block(x,num_filters,block_id,stack)
        num_filters *= 2
    x = Flatten()(x)
    x = Dense(1)(x)
    outputs = Activation('sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [4]:
input=(20,20,1)
CNN=creation_model(input,14,2)
# Ottieni la rappresentazione JSON del modello
modello_json = CNN.to_json()

# Salva la rappresentazione JSON del modello in un file
with open('CNN.json', 'w') as f:
    f.write(modello_json)
plot_model(CNN, to_file='CNN.png',show_shapes=True, show_layer_names=True)

with open('CNN_summary.txt', 'w') as f:
    CNN.summary(print_fn=lambda x: f.write(x + '\n'))

2024-07-11 14:18:54.652559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-11 14:18:54.733816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-11 14:18:54.734212: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-11 14:18:54.735329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFl

CNN_ResNet

In [5]:
def residual_block(x, filters,block_id=0,stack_id=0, kernel_size=3, stride=1):
    y = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    y = Activation('relu')(y)
    y = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(y)
    if block_id==0 and stack_id!=0:
        x = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(x)
    y = Add()([x, y])
    y = Activation('relu')(y)
    return y
def resnet(input_shape, depth, num_classes=2):
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    inputs = Input(shape=input_shape)
    x = Conv2D(num_filters, kernel_size=3, strides=1, padding='same')(inputs)
    for stack in range(3):
        for res_block in range(num_res_blocks):
           x = residual_block(x, num_filters,res_block,stack)
        num_filters *= 2
    x=Flatten()(x)
    x = Dense(1)(x)
    outputs = Activation('sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [7]:
input=(20,20,1)
CNN_ResNet=resnet(input,14,2)

# Ottieni la rappresentazione JSON del modello
modello_json = CNN_ResNet.to_json()

# Salva la rappresentazione JSON del modello in un file
with open('ResNet_CNN.json', 'w') as f:
    f.write(modello_json)
plot_model(CNN_ResNet, to_file='ResNet_CNN.png',show_shapes=True, show_layer_names=True)

with open('ResNet_CNN_summary.txt', 'w') as f:
    CNN_ResNet.summary(print_fn=lambda x: f.write(x + '\n'))

Inception

In [8]:
def initial_conv(input_img):
    x = Conv2D(8, (2, 2), activation='relu', padding='same')(input_img)
    x = BatchNormalization(axis=3)(x)
    x = MaxPooling2D((2, 2), strides=2)(x)
    x = Conv2D(8, (2, 2), activation='relu', padding='same')(x)
    x = BatchNormalization(axis=3)(x)
    return x

def Conv_2D_Block(x, model_width, kernel, strides=(1, 1), padding="same"):
    # 2D Convolutional Block with BatchNormalization
    x = tf.keras.layers.Conv2D(model_width, kernel, strides=strides, padding=padding, kernel_initializer="he_normal")(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def Inceptionv1_Module(inputs, filterB1_1, filterB2_1, filterB2_2, filterB3_1, filterB3_2, filterB4_1, i):
    # Inception Block i
    branch1x1 = Conv_2D_Block(inputs, filterB1_1, (1, 1), padding='valid')
    branch1x1 = Conv_2D_Block(inputs, filterB1_1, (1, 1), padding='valid')

    branch3x3 = Conv_2D_Block(inputs, filterB2_1, (1, 1), padding='valid')
    branch3x3 = Conv_2D_Block(branch3x3, filterB2_2, (3, 3))

    branch5x5 = Conv_2D_Block(inputs, filterB3_1, (1, 1), padding='valid')
    branch5x5 = Conv_2D_Block(branch5x5, filterB3_2, (5, 5))

    out = tf.keras.layers.concatenate([branch1x1, branch3x3, branch5x5], axis=-1, name='Inception_Block_'+str(i))
    return out

def final_block(input):
    # Average pooling
    x = AveragePooling2D((3,3), strides=1)(input)

    # Flatten
    x = Flatten()(x)

    # Fully connected layer
    x = Dense(1, activation='sigmoid')(x)
    return x

In [11]:
inputs = Input(shape=(20, 20,1))
x = initial_conv(inputs)
x = Inceptionv1_Module(x, 16, 16, 16, 16, 16, 16, 3)
x = Inceptionv1_Module(x, 32, 32, 32, 32, 32, 32, 4)
x = Inceptionv1_Module(x, 64, 64, 64, 64,  64, 64, 5)
x = Inceptionv1_Module(x, 64, 64, 64, 64,  64, 64, 6)
output = final_block(x)

v1=Model(inputs,output)

# Ottieni la rappresentazione JSON del modello
modello_json = v1.to_json()
# Salva la rappresentazione JSON del modello in un file
with open('V1_CNN.json', 'w') as f:
    f.write(modello_json)
    
plot_model(v1, to_file='V1_CNN.png',show_shapes=True, show_layer_names=True)

with open('V1_CNN_summary.txt', 'w') as f:
    v1.summary(print_fn=lambda x: f.write(x + '\n'))